In [13]:
import gzip
from glob import glob
import os
import pandas as pd
from collections import Counter

# Load the reference sequences (positions 16-45)
def load_references(reference_seq_file):
    refs = {}
    with open(reference_seq_file, 'r') as f:
        for line1 in f:
            line2 = f.readline()
            refs[line2[15:45]] = line1.strip()[1:]  # Extract positions 16-45
    return refs


def count_all_in_fastq(fastq_gz_file,start,end):
    # count all unique sequences in each fastq file
    # skipping the begining and end of each read
    counts={}
    with gzip.open(fastq_gz_file, 'rb') as f:
        line_nb = 0
        for line in f:
            line_nb = line_nb + 1
            if(line_nb % 4 == 2):
                seq = line.decode().strip()
                if (not ('N' in seq)) and len(seq) > end: # read long enough, does not contain N
                    seq = seq[start:end]
                    if not seq in counts:
                        counts[seq] = 1
                    else:
                        counts[seq] = counts[seq] + 1
    return counts

def count_ref_in_fastq(fastq_gz_file,start,end,refs):
    # count only reference sequence in each fastq file
    # skipping the begining and end of each read
    counts={}
    for seq in refs:
        counts[seq] = 0
    # total number of reads in the file
    total_read_count = 0
    # number of matched reads in the file
    match_read_count = 0 
    with gzip.open(fastq_gz_file, 'rb') as f:
        line_nb = 0
        for line in f:
            line_nb = line_nb + 1
            if(line_nb % 4 == 2):
                total_read_count = total_read_count + 1
                seq = line.decode().strip()[start:end]
                if seq in counts:
                    counts[seq] = counts[seq] + 1
                    match_read_count = match_read_count + 1
    return total_read_count,match_read_count,counts

In [14]:
from glob import glob
import os
import pandas as pd
from collections import Counter

# Function to process all samples (single-end files only)
def process_samples(folder, refs, output_csv, start=0, end=30):
    # Initialize the output table with reference sequences
    counts_df = pd.DataFrame({'Seq': list(refs.keys()), 'Name': list(refs.values())})

    # Iterate through cell types and replicates
    for cell in ['hek-gDNA', 'hek-RNA', 'sk-gDNA', 'sk-RNA', 'aso-lib']:
        for rep in ['1', '2', '3']:  # Iterate over replicates
            # File matching for R1 files
            sample_r1_files = glob(os.path.join(folder, f"{cell}{rep}_R1.fastq.gz"))

            # Check if R1 files exist
            if sample_r1_files:
                # Accumulate counts for all FASTQ files for the current sample
                sample_counts = Counter()
                total_reads = 0
                matched_reads = 0

                for file in sample_r1_files:
                    total, matched, cts = count_ref_in_fastq(file, start, end, refs)
                    total_reads += total
                    matched_reads += matched
                    sample_counts.update(cts)

                    print(f"{cell}{rep} | File: {os.path.basename(file)} | Total Reads: {total} | "
                          f"Matched Reads: {matched} | Match Ratio: {matched / total:.2%}")

                # Update counts for the current sample in the DataFrame
                counts_df[f"{cell}{rep}"] = counts_df['Seq'].map(sample_counts).fillna(0).astype(int)

    # Save the counts to a CSV file
    counts_df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

# Paths and parameters
reference_seq_file = '/home/yg2895/media/protein/yg2895/ASO-lib-20250102/haplo-utr3-oligo-pool.fa'
fastq_folder = '/home/yg2895/media/protein/sequencing_data2/20250102_AV100007_20250102-aso-lib-gy/fastq'
output_csv = 'counts-aso.csv'

# Load references and process samples
refs = load_references(reference_seq_file)
process_samples(fastq_folder, refs, output_csv)



hek-gDNA1 | File: hek-gDNA1_R1.fastq.gz | Total Reads: 40799299 | Matched Reads: 39921979 | Match Ratio: 97.85%
hek-gDNA2 | File: hek-gDNA2_R1.fastq.gz | Total Reads: 36674142 | Matched Reads: 35866908 | Match Ratio: 97.80%
hek-gDNA3 | File: hek-gDNA3_R1.fastq.gz | Total Reads: 43857872 | Matched Reads: 42936728 | Match Ratio: 97.90%
hek-RNA1 | File: hek-RNA1_R1.fastq.gz | Total Reads: 38185731 | Matched Reads: 37116723 | Match Ratio: 97.20%
hek-RNA2 | File: hek-RNA2_R1.fastq.gz | Total Reads: 36325027 | Matched Reads: 35320056 | Match Ratio: 97.23%
hek-RNA3 | File: hek-RNA3_R1.fastq.gz | Total Reads: 35708490 | Matched Reads: 34556538 | Match Ratio: 96.77%
sk-gDNA1 | File: sk-gDNA1_R1.fastq.gz | Total Reads: 42869191 | Matched Reads: 41763799 | Match Ratio: 97.42%
sk-gDNA2 | File: sk-gDNA2_R1.fastq.gz | Total Reads: 35818273 | Matched Reads: 34932492 | Match Ratio: 97.53%
sk-gDNA3 | File: sk-gDNA3_R1.fastq.gz | Total Reads: 43676344 | Matched Reads: 41427481 | Match Ratio: 94.85%
sk-R